<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/Pycaret_%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%A0%9C%EA%B1%B0_MIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pycaret MI 결측치 제거**

[입력 파일]
1. 유전형인코딩_결측치_행_제거.csv
2. 2016년 표현형 데이터.xlsx
3. 상위N개_MI.csv

In [ ]:
# ✅ 라이브러리 설치
!pip install pycaret lightgbm xgboost catboost openpyxl --quiet
!pip install numpy==1.24.4 --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.

In [ ]:
# ✅ 라이브러리
from pycaret.regression import setup, compare_models, pull
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# ✅ 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving 유전형인코딩_결측치_행_제거.csv to 유전형인코딩_결측치_행_제거.csv
Saving 상위10000개_MI.csv to 상위10000개_MI.csv
Saving 상위5000개_MI.csv to 상위5000개_MI.csv
Saving 상위3000개_MI.csv to 상위3000개_MI.csv
Saving 상위1000개_MI.csv to 상위1000개_MI.csv
Saving 상위500개_MI.csv to 상위500개_MI.csv
Saving 상위100개_MI.csv to 상위100개_MI.csv
Saving 2016년 표현형 데이터.xlsx to 2016년 표현형 데이터.xlsx


In [ ]:
geno_path = "유전형인코딩_결측치_행_제거.csv"
pheno_path = "2016년 표현형 데이터.xlsx"

snp_files = {
    "Top 100": "상위100개_MI.csv",
    "Top 500": "상위500개_MI.csv",
    "Top 1000": "상위1000개_MI.csv",
    "Top 3000": "상위3000개_MI.csv",
    "Top 5000": "상위5000개_MI.csv",
    "Top 10000": "상위10000개_MI.csv"
}

In [ ]:
# ✅ 데이터 로딩
geno_df = pd.read_csv(geno_path, index_col=0)  # SNP x 샘플
pheno_df = pd.read_excel(pheno_path)
pheno_df.columns = pheno_df.columns.str.strip()
geno_df_t = geno_df.T  # 전치: 샘플 x SNP
geno_df_t.index.name = "Genotype"
pheno_df.set_index("Genotype", inplace=True)

# ✅ 결과 저장용 리스트
all_results = []

In [ ]:
# ✅ 표현형별 + SNP 수별 모델 성능 비교
for label, file_path in snp_files.items():
    snp_df = pd.read_csv(file_path)
    for trait in snp_df["Phenotype"].unique():
        try:
            top_snps = snp_df[snp_df["Phenotype"] == trait]["SNP"].tolist()
            geno_subset = geno_df_t[top_snps].copy()
            geno_subset["target"] = pheno_df[trait]

            # PyCaret 환경 구성
            setup(data=geno_subset, target="target", session_id=42, verbose=False, use_gpu=False)

            # 모델 비교 (4개만)
            compare_models(include=["rf", "xgboost", "lightgbm", "catboost"], n_select=4)
            result_df = pull()
            result_df["SNP 개수"] = label
            result_df["표현형"] = trait

            print(f"✅ 표현형: {trait} | SNP 수: {label}")
            display(result_df)

            all_results.append(result_df)

        except Exception as e:
            print(f"⚠️ 오류 발생 - 표현형: {trait}, SNP: {label}, 원인: {e}")
            continue

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,32.9834,2297.0657,46.6370,0.1732,0.5774,0.6314,1.7640
lightgbm,Light Gradient Boosting Machine,34.3042,2400.4134,47.6824,0.1727,0.5974,0.6720,0.1270
rf,Random Forest Regressor,33.3996,2363.2626,47.3235,0.1135,0.5745,0.6387,0.9120
xgboost,Extreme Gradient Boosting,33.2493,2314.4538,46.6816,0.1123,0.6252,0.6236,0.2330


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

✅ 표현형: 과중 (g) | SNP 수: Top 100


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
catboost,CatBoost Regressor,32.9834,2297.0657,46.6370,0.1732,0.5774,0.6314,1.764,Top 100,과중 (g)
lightgbm,Light Gradient Boosting Machine,34.3042,2400.4134,47.6824,0.1727,0.5974,0.6720,0.127,Top 100,과중 (g)
rf,Random Forest Regressor,33.3996,2363.2626,47.3235,0.1135,0.5745,0.6387,0.912,Top 100,과중 (g)
xgboost,Extreme Gradient Boosting,33.2493,2314.4538,46.6816,0.1123,0.6252,0.6236,0.233,Top 100,과중 (g)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,6.3088,83.6775,8.7422,0.2543,0.1798,0.1420,0.3430
catboost,CatBoost Regressor,6.3696,84.5866,8.7763,0.2445,0.1797,0.1431,2.0460
lightgbm,Light Gradient Boosting Machine,6.6224,89.6717,9.0335,0.2313,0.1840,0.1472,0.1170
xgboost,Extreme Gradient Boosting,6.9191,101.3217,9.6854,0.0495,0.1978,0.1554,0.1430


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
catboost,CatBoost Regressor,0.6302,0.6801,0.8114,0.3576,0.1564,0.1575,193.031,Top 10000,당도 (%)
rf,Random Forest Regressor,0.6248,0.6994,0.8179,0.3308,0.1567,0.1593,12.322,Top 10000,당도 (%)
lightgbm,Light Gradient Boosting Machine,0.6956,0.7929,0.8810,0.2222,0.1665,0.1733,3.850,Top 10000,당도 (%)
xgboost,Extreme Gradient Boosting,0.7030,0.8867,0.9072,0.1770,0.1737,0.1786,9.743,Top 10000,당도 (%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0827,0.0108,0.1017,0.2643,0.0635,0.1451,192.9790
rf,Random Forest Regressor,0.0812,0.0109,0.1022,0.2600,0.0636,0.1423,12.0510
xgboost,Extreme Gradient Boosting,0.0886,0.0123,0.1076,0.1809,0.0670,0.1547,9.7010
lightgbm,Light Gradient Boosting Machine,0.0894,0.0125,0.1099,0.1302,0.0684,0.1562,3.5960


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

✅ 표현형: 과실경도 (kg) | SNP 수: Top 10000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
catboost,CatBoost Regressor,0.0827,0.0108,0.1017,0.2643,0.0635,0.1451,192.979,Top 10000,과실경도 (kg)
rf,Random Forest Regressor,0.0812,0.0109,0.1022,0.2600,0.0636,0.1423,12.051,Top 10000,과실경도 (kg)
xgboost,Extreme Gradient Boosting,0.0886,0.0123,0.1076,0.1809,0.0670,0.1547,9.701,Top 10000,과실경도 (kg)
lightgbm,Light Gradient Boosting Machine,0.0894,0.0125,0.1099,0.1302,0.0684,0.1562,3.596,Top 10000,과실경도 (kg)


In [ ]:
# ✅ 결과 통합 및 저장
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    final_df.to_csv("모델성능비교_MI_전체정리.csv", index=False)
    print("✅ 결과 저장 완료 → 모델성능비교_MI_전체정리.csv")
    display(final_df.head(10))
else:
    print("❌ 저장할 결과가 없습니다.")

✅ 결과 저장 완료 → 모델성능비교_MI_전체정리.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,CatBoost Regressor,32.9834,2297.0657,46.6370,0.1732,0.5774,0.6314,1.764,Top 100,과중 (g)
1,Light Gradient Boosting Machine,34.3042,2400.4134,47.6824,0.1727,0.5974,0.6720,0.127,Top 100,과중 (g)
2,Random Forest Regressor,33.3996,2363.2626,47.3235,0.1135,0.5745,0.6387,0.912,Top 100,과중 (g)
3,Extreme Gradient Boosting,33.2493,2314.4538,46.6816,0.1123,0.6252,0.6236,0.233,Top 100,과중 (g)
4,Random Forest Regressor,6.3088,83.6775,8.7422,0.2543,0.1798,0.1420,0.343,Top 100,과장 (mm)
5,CatBoost Regressor,6.3696,84.5866,8.7763,0.2445,0.1797,0.1431,2.046,Top 100,과장 (mm)
6,Light Gradient Boosting Machine,6.6224,89.6717,9.0335,0.2313,0.1840,0.1472,0.117,Top 100,과장 (mm)
7,Extreme Gradient Boosting,6.9191,101.3217,9.6854,0.0495,0.1978,0.1554,0.143,Top 100,과장 (mm)
8,Light Gradient Boosting Machine,8.6412,131.5273,11.3261,0.3946,0.2208,0.1846,0.067,Top 100,과폭 (mm)
9,Random Forest Regressor,9.0269,141.5930,11.7383,0.3441,0.2297,0.1929,0.181,Top 100,과폭 (mm)


In [ ]:
# 📌 Colab 전용 성능 요약 코드 (R² 기준 요약 3종)

import pandas as pd

# # 👉 파일 업로드
# from google.colab import files
# uploaded = files.upload()  # 모델성능비교_전체정리.csv 업로드

# 👉 파일 로드
df = pd.read_csv("모델성능비교_MI_전체정리.csv")

# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["R2"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 표현형별 최고 R² 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 R²")
display(mean_r2_by_snp)

print("📌 모델별 평균 R²")
display(mean_r2_by_model)

📌 표현형별 최고 R² 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,CatBoost Regressor,0.0776,0.0103,0.0990,0.3053,0.0617,0.1352,18.042,Top 1000,과실경도 (kg)
1,CatBoost Regressor,5.9504,77.4093,8.4322,0.2904,0.1770,0.1368,65.120,Top 3000,과장 (mm)
2,CatBoost Regressor,25.9638,1295.4360,35.6553,0.4675,0.5213,0.5706,51.213,Top 3000,과중 (g)
3,CatBoost Regressor,7.1844,92.2746,9.5173,0.5437,0.1929,0.1589,53.314,Top 3000,과폭 (mm)
4,Random Forest Regressor,0.5318,0.5467,0.7293,0.1789,0.1290,0.1189,2.851,Top 3000,과피두께 (mm)
5,CatBoost Regressor,0.6136,0.6428,0.7901,0.3896,0.1540,0.1546,54.433,Top 3000,당도 (%)


📌 SNP 개수별 평균 R²


,SNP 개수,평균 R²
0,Top 100,0.172225
1,Top 1000,0.243442
2,Top 10000,0.204646
3,Top 3000,0.254767
4,Top 500,0.196208
5,Top 5000,0.229012


📌 모델별 평균 R²


,Model,평균 R²
0,CatBoost Regressor,0.302050
1,Extreme Gradient Boosting,0.084772
2,Light Gradient Boosting Machine,0.207333
3,Random Forest Regressor,0.272711
